# Importamos las librerías necesarias

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import expit
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold

# Esquema del código

R -> rating matrix \
k -> número de factores latentes \
gamma -> learning rate \
eta -> Regularization\
m -> número de iteraciones\
S -> conjunto de valoraciones posibles\
logit -> 1 / (1 + np.exp(-x))

![Ejemplo de imagen PNG](Fact_Bernoulli_Pseudocodigo.png)

# Implementación del algoritmo

In [ ]:
class BernoulliMF_new_v2:
    def __init__(self, ratings_matrix, num_users, num_items, num_factors, lr=0.02, reg=0.1, num_epochs=100):
        self.ratings_matrix = ratings_matrix
        self.num_users = num_users
        self.num_items = num_items
        self.num_factors = num_factors
        self.lr = lr
        self.reg = reg
        self.num_epochs = num_epochs
        #self.scores = scores
        
    def _logit(self, x):
        return expit(x)
    
    def elementos_no_nulos(self,lista):
        no_nulos= []
        for i in range(len(lista)):
            if lista[i] !=-1:
                no_nulos.append(i)
        return no_nulos

    def usuarios_no_nulos(self,ratings_matrix, item_index):
        no_nulos= []
        contador = 0
        for lista in ratings_matrix:
            if lista[item_index] != -1:
                no_nulos.append(contador)
            contador +=1
        return no_nulos

    def fit(self):
        
        # Inicializamos las matrices de factores latentes para el usuario y el ítem para cada valoración
        self.user_factors_0 = np.random.randn(self.num_users, self.num_factors)
        self.item_factors_0 = np.random.randn(self.num_items, self.num_factors)
        
        self.user_factors_1 = np.random.randn(self.num_users, self.num_factors)
        self.item_factors_1 = np.random.randn(self.num_items, self.num_factors)
        
        for iteracion in range(self.num_epochs):
            for score in [0,1]:
                for u in range(self.num_users):
                    nonzero_indices = self.elementos_no_nulos(self.ratings_matrix[0])
                    Alpha_0 = np.zeros(self.num_factors, dtype=np.float32)
                    Alpha_1 = np.zeros(self.num_factors, dtype=np.float32)
                    for i in nonzero_indices: 
                        if self.ratings_matrix[u][i] == score:
                            if score == 1:
                                Alpha_1 += (1- self._logit(np.dot(self.user_factors_1[u], self.item_factors_1[i])))*self.item_factors_1[i]
                            else:
                                Alpha_0 += (1- self._logit(np.dot(self.user_factors_0[u], self.item_factors_0[i])))*self.item_factors_0[i]
                        else:
                            if score == 1:
                                Alpha_1 += self._logit(np.dot(self.user_factors_1[u], self.item_factors_1[i]))*self.item_factors_1[i]
                            else:
                                Alpha_0 += self._logit(np.dot(self.user_factors_0[u], self.item_factors_0[i]))*self.item_factors_0[i]

                    if score == 0:
                        self.user_factors_0[u] += self.lr*(Alpha_0 - self.reg *self.user_factors_0[u])
                    else:
                        self.user_factors_1[u] += self.lr*(Alpha_1 - self.reg *self.user_factors_1[u])




                for i in range(self.num_items):
                    users = self.usuarios_no_nulos(self.ratings_matrix, i)
                    Theta_0 = np.zeros(self.num_factors, dtype=np.float32)
                    Theta_1 = np.zeros(self.num_factors, dtype=np.float32)
                    for u in users: 
                        if self.ratings_matrix[u][i] == score:
                            if score == 1:
                                Theta_1 +=(1- self._logit(np.dot(self.user_factors_1[u], self.item_factors_1[i])))*self.user_factors_1[u]
                            else:
                                Theta_0 +=(1- self._logit(np.dot(self.user_factors_0[u], self.item_factors_0[i])))*self.user_factors_0[u]
                        else:
                            if score == 1:
                                Theta_1 += self._logit(np.dot(self.user_factors_1[u], self.item_factors_1[i]))*self.user_factors_1[u]
                            else:
                                Theta_0 += self._logit(np.dot(self.user_factors_0[u], self.item_factors_0[i]))*self.user_factors_0[u]

                    if score == 1:
                        self.item_factors_1[i] +=self.lr*(Theta_1 - self.reg *self.item_factors_1[i])
                    else:
                        self.item_factors_0[i] +=self.lr*(Theta_0 - self.reg *self.item_factors_0[i])
                        
                        
                        
    def get_prob_vector(self,u,i):
        pred1 =self._logit(np.dot(self.user_factors_1[u], self.item_factors_1[i]))
        pred0 =self._logit(np.dot(self.user_factors_0[u], self.item_factors_0[i]))
        suma = pred0 +pred1
        pred0 = pred0 /suma
        pred1 = pred1 /suma
        return pred0,pred1    
    
    def predict(self,u,i):
        pred1 =self._logit(np.dot(self.user_factors_1[u], self.item_factors_1[i]))
        pred0 =self._logit(np.dot(self.user_factors_0[u], self.item_factors_0[i]))
        suma = pred0 +pred1
        pred0 = pred0 /suma
        pred1 = pred1 /suma
        if pred0> pred1:
            return 0
        else:
            return 1
    
                        
        

# Importamos los datos 

In [ ]:
df_datos = pd.read_excel('C:/Users/amidonga/Documents/TFG/Datos_Sinteticos_CON_TRAMITES.xlsx')

## Rellenamos los valores nulos con -1

In [ ]:
df_datos.fillna(-1,inplace= True)

### Calculamos los n usuarios más parecidos a un usuario concreto

In [ ]:
def calcular_usuarios_similares(df, usuario_concreto, n):
    # Filtrar el dataframe por las características relevantes para el cálculo de similitud
    features = ['Edad', 'Procedencia', 'Sexo', 'Situación de dependencia', 'Sector económico',
                'Renta anual neta', 'Estado civil', 'Número de hijos']
    df_filt = df[features].copy()

    # Codificar las variables categóricas utilizando one-hot encoding
    df_encoded = pd.get_dummies(df_filt)
    # Obtener el índice del usuario concreto
    idx_usuario_concreto = df_filt.index[df_filt.index == usuario_concreto]
    
    # Calcular la similitud del coseno entre el usuario concreto y todos los demás usuarios
    similarities = cosine_similarity(df_encoded.iloc[idx_usuario_concreto], df_encoded)

    # Obtener los índices de los usuarios más similares (excluyendo al usuario concreto)
    similar_users_indices = np.argsort(similarities[0])[::-1][:n]
    
    # Agregar el índice del usuario concreto al conjunto de usuarios similares
    #similar_users_indices = np.concatenate((idx_usuario_concreto, similar_users_indices))

    # Obtener los datos de los usuarios más similares
    similar_users = df.loc[similar_users_indices]

    return similar_users


### Función para preparar la matriz de valoraciones a partir del dataframe de datos

In [ ]:
def crear_ratings_matrix(df):    
    # Obtén el número de filas y columnas
    num_rows, num_cols = df.shape
    # Crea una matriz de tamaño adecuado, inicializada con None
    ratings_matrix = np.full((num_rows, num_cols),None, dtype=object)

    # Itera sobre las filas del dataframe
    for row_idx, row in df.iterrows():
        # Itera sobre las columnas del dataframe
        for col_idx, col_label in enumerate(df.columns):
            # Obtiene el valor de la celda
            value = row[col_label]
            # Si el valor no es NaN, colócalo en la matriz
            if not pd.isnull(value):
                ratings_matrix[row_idx, col_idx] = int(value)
        
    return ratings_matrix


# Ejemplo de uso concreto

In [ ]:
### BIEN la valoración es 1 ->0


usuario= 90
item = 1 #empieza en 0 para predict 1=0
# Calculamos los 200 usuarios más similares al usuario elegido.
usuarios_similares = calcular_usuarios_similares(df_datos, usuario_concreto = usuario, n=200)

# Creamos la matriz de valoraciones de estos usuarios
df_ratings_matrix = usuarios_similares[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']]
df_ratings_matrix.reset_index(inplace=True)
df_ratings_matrix=df_ratings_matrix[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']]
ratings_matrix_u=crear_ratings_matrix(df_ratings_matrix)


# Ajustamos el modelo
bemf_model = BernoulliMF_new_v2(ratings_matrix=ratings_matrix_u, num_users=200, num_items=7, num_factors=10, lr=0.02, reg=0.1, num_epochs=100)
    
# Actualizaciones del proceso
bemf_model.fit()

# predicción 
prediction = bemf_model.predict(0, item-1)
prediction


# print('La predicción para el usuario',usuario,'y el item',item,'es la valoración',prediction-1,'con probabilidad',probabilidad)

# Función para automatizar la predicción 

In [ ]:
def prediccion(usuario,item, df_para_usuarios_similares): # usuarios de 0 a n (indices tabla).  Item 0 se corresponde con el 1
    # Tomamos los 200 usuarios más similares a él
    usuarios_similares = calcular_usuarios_similares(df_para_usuarios_similares, usuario_concreto = usuario, n=350)
    df_ratings_matrix = usuarios_similares[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']]
    df_ratings_matrix.reset_index(inplace=True)
    df_ratings_matrix=df_ratings_matrix[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']]
    
    # Creamos la matriz asociada a este dataframe
    ratings_matrix_u=crear_ratings_matrix(df_ratings_matrix)
    
    # Ajustamos el modelo
    possible_scores = [1,2]
    user_ids = list(range(0,350))  # Lista de ID de usuarios
    item_ids = [1,2,3,4,5,6,7]  # Lista de ID de elementos
    num_factors = 10
    num_iters = 100
    learning_rate = 0.02 
    regularisation = 0.1
    seed = 42

    bemf_model = BernoulliMF_new_v2(ratings_matrix=ratings_matrix_u, num_users=350, num_items=7, num_factors=10, lr=0.02, reg=0.1, num_epochs=100)
    
    # Actualizaciones del proceso
    bemf_model.fit()
    
    # predicción 
    prediction = bemf_model.predict(0, item -1)
    
    return prediction

## Estuadiamos el rendimiento del modelo

### Accuracy True/False

Calculamos la mtriz de valoraciones general

In [ ]:
ratings_matrix_general = crear_ratings_matrix(df_datos[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']])

Estudiamos el redimiento calculando la predicción sobre las valoraciones de las que ya tenemos datos

## Automatizamos la evaluación del modelo

In [ ]:
def evaluar_modelo(ratings_matrix_general,df_para_usuarios_similares, n_usuarios): # u en 0-n-1
    accuracies = []
    confusion_matrix = np.zeros((2, 2), dtype=int)
    TP= 0
    FP= 0
    FN = 0
    for u in range(n_usuarios):
        for i in range(7):
            valor_real = ratings_matrix_general[u][i]
            if valor_real != -1:
                prediction =prediccion(u,i+1,df_para_usuarios_similares)
                print('valor',valor_real)
                print('prediccion',prediction)
                accuracies.append(prediction==valor_real)
                confusion_matrix[valor_real, prediction] += 1
                if (valor_real == 1) &(prediction == 1):
                    TP +=1
                elif (valor_real == 0) &(prediction == 1):
                    FP +=1
                elif(valor_real == 1) &(prediction == 0):
                    FN +=1

    confusion_matrix_percent = np.round(confusion_matrix.astype(float) / confusion_matrix.sum(axis=1, keepdims=True) * 100, 2)
    acc = accuracies.count(True)*100/len(accuracies)
    
    # F1 score
    F1 = TP/(TP+0.5*(FP+FN))
    # Configurar el formato de impresión
    np.set_printoptions(formatter={'float': lambda x: "{:.2f}".format(x)})

    return acc,confusion_matrix_percent,F1

# Evaluación del sistema sobre un muestreo aleatorio
Para seleccionar un subconjunto aleatorio, utilizamos la función sample() de pandas.El parámetro "n" indica el número de muestras aleatorias que deseas seleccionar

In [ ]:
df_datos_subset = df_datos.sample(n=100).reset_index()

Evaluamos el modelo sobre el subconjunto de datos seleccionado. Utilizamos aún así el conjunto de datos completo para calcular los usuarios similares de cada usuario.

In [ ]:
ratings_matrix_general_subset = crear_ratings_matrix(df_datos_subset[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']])

In [ ]:
evaluar_modelo(ratings_matrix_general=ratings_matrix_general_subset,df_para_usuarios_similares = df_datos, n_usuarios=100)

# Evaluación sobre los usuarios más activos
Calculamos el número de evaluaciones que ha realizado cada usuario. Tomaremos solo aquellos usuarios que hayan valorado más de tres items para llevar a cabo la evaluación de nuestro modelo.

In [ ]:
# Añadimos una variable que indica el número de evaluaciones realizadas por cada usuario
df_datos['num_evaluations'] = df_datos[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']].notnull().sum(axis=1)

# Tomamos solo aquellos usuarios con más de tres valoraciones.
df_datos_mas3_valoraciones =df_datos[df_datos['num_evaluations']>3][['Comunidad Autónoma', 'Edad', 'Procedencia', 'Sexo',
       'Situación de dependencia', 'Sector económico', 'Renta anual neta',
       'Estado civil', 'Número de hijos', 'T.1.', 'T.2.', 'T.3.', 'T.4.',
       'T.5.', 'T.6.', 'T.7.']].reset_index()

# Tomamos un subconjunto de los usuarios más activos
df_activos_subset = df_datos_mas3_valoraciones.sample(n=100).reset_index()


Evaluamos el modelo sobre el subconjunto de datos seleccionado. Utilizamos el conjunto de usuarios más activos para calcular los usuarios similares.

In [ ]:
ratings_matrix_activos_subset = crear_ratings_matrix(df_activos_subset[['T.1.', 'T.2.', 'T.3.', 'T.4.','T.5.', 'T.6.', 'T.7.']])

In [ ]:
evaluar_modelo(ratings_matrix_general=ratings_matrix_activos_subset,df_para_usuarios_similares = df_datos_mas3_valoraciones, n_usuarios=100)